In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

# Plotting style
sns.set_style('darkgrid')
sns.set_theme(font_scale=1.)

# Define parameters
seed = 67

In [ ]:
# Initial data clean
df = pd.read_csv('data/SAHeart.csv')
df['chd'] = pd.Categorical(df['chd'])
df['famhist'] = pd.Categorical(df['famhist'].map({'Absent':0, 'Present':1})) # One-of-K coding

# Filter outliers if we need to??
#df = df[~( (df[''] > 2) | (df[''] > 1) )]

df.head()

In [ ]:
# What to predict for Regression
X = df.drop(columns=['ldl', 'chd'])
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
# Target
y = df['ldl'] 


test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=seed)